### Import packages and log in to plotly

In [1]:
import numpy as np
import pandas as pd
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as ff

import terminal_optimization.visualisation        as visualisation
import terminal_optimization.forecast             as forecast
import terminal_optimization.financial_analysis   as financial

In [2]:
# Log in to Plotly servers
import plotly
#plotly.tools.set_credentials_file(username='wijzermans', api_key='FKGDvSah3z5WCNREBZEq')
plotly.tools.set_credentials_file(username='wijnandijzermans', api_key='xeDEwwpCK3aLLR4TIrM9')

# (*) To communicate with Plotly's server, sign in with credentials file
import plotly.plotly as py  

# (*) Useful Python/Plotly tools
import plotly.tools as tls   

# (*) Graph objects to piece together plots
from plotly.graph_objs import *

import matplotlib.pyplot as plt # (*) import matplotlib

%matplotlib inline

### Traffic projections

In [9]:
###################################################################################################
# Traffic projections on which estimate is based
###################################################################################################

# Import commodities from package
maize = forecast.bulk_commodities(**forecast.maize_data)

# Wheat - Probabilistic demand
historic_demand_maize = [1000000,1200000,1180000,1210000,1250000]
rate_maize  = 1.02
mu_maize    = 0.01          # avg bonus rate added to base rate (% points)  - input for random method
sigma_maize = 0.025         # standard deviation of bonus rate (% points)   - input for random method

# Create traffic simulations
iterations = 1001
start_year = 2018
simulation_window = 20
traffic_simulations = []
for iteration in range(iterations):
    maize.random_scenario(start_year, simulation_window, historic_demand_maize  , rate_maize, mu_maize, sigma_maize)
    traffic_simulations.append(maize.demand)

# Create traffic projections
traffic_projections = []
for iteration in range(iterations):
    maize.random_scenario(start_year, simulation_window, historic_demand_maize  , rate_maize, mu_maize, sigma_maize)
    traffic_projections.append(maize.demand)

### Traffic Scenario (based on median NPQ)

In [10]:
# Calculate the net present quantity of each projection
project_WACC = 0.09
WACC_factor = []
for year in range (len(historic_demand_maize)):
    WACC_factor.append(1.0)
for year in range (simulation_window):
    WACC_factor.append(1/((1+project_WACC)**(year)))

traffic_matrix = np.zeros(shape=(len(traffic_projections), 2))
for i in range(len(traffic_projections)):
    traffic_matrix[i,0] = i
    present_value_volume = []
    for j in range(len(traffic_projections[i])):
        present_value_volume.append(traffic_projections[i][j]*WACC_factor[j])
    traffic_matrix[i,1] = int(np.sum(present_value_volume))

df=pd.DataFrame(traffic_matrix, columns=['Iteration','NPQ'])

# Select the median net present quantity
median_iteration = df.loc[df['NPQ']==np.median(df['NPQ'])].index[0]
maize.demand = traffic_projections[median_iteration]

### Visualizations
- Traffic projections (exc scenario)
- Traffic projections vs. traffic simulations
- Traffic projections (inc scenario)
- Single projection to net present quantity
- NPQ distribution
- Timing of investments

#### Traffic projections (exc scenario)

In [ ]:
# Visualize each traffic projections
data = []
for i in range(len(traffic_projections)):
    data.append(
        go.Scatter(
            x = maize.years,
            y = traffic_projections[i],
            name = 'Iteration '+ str(i),
            showlegend=False,
            line = dict(
                color = ('rgb(215, 215, 215)'),
                width = 2)))

# Edit the layout
layout = dict(
            title = 'Traffic projections',
            yaxis = dict(title = 'Maize throughput (t/year)'),
            showlegend=False,
            annotations=[
                dict(
                    x=1,
                    y=-0.12,
                    showarrow=False,
                    text='Source: Own work',
                    xref='paper',
                    yref='paper'),
                dict(
                    x=2018.7,
                    y=1,
                    showarrow=False,
                    text='Projected traffic',
                    yref='paper',
                    textangle=90,
                    font=dict(
                        size=13)),
                dict(
                    x=2017.3,
                    y=1,
                    showarrow=False,
                    text='Historic traffic',
                    yref='paper',
                    textangle=90,
                    font=dict(
                        size=13))],
            shapes=[
                dict(
                    type='line',
                    x0=2018,
                    y0=-0.05,
                    x1=2018,
                    y1=1.05,
                    yref='paper',
                    line=dict(
                        color='rgb(60, 60, 60)',
                        width=3,
                        dash='dashdot'))]
)

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='Traffic projections')

#### Traffic projections vs. traffic simulations

In [ ]:
# Visualize each traffic projections
data = []
for i in range(len(traffic_projections)):
    if i == 0:
        data.append(
            go.Scatter(
                x = maize.years,
                y = traffic_simulations[i],
                name = 'Traffic<br>simulations',
                line = dict(
                    color = ('rgb(200, 200, 200)'),
                    width = 1.5)))

    data.append(
        go.Scatter(
            x = maize.years,
            y = traffic_simulations[i],
            name = 'Simulation '+ str(i),
            showlegend=False,
            line = dict(
                color = ('rgba(55, 83, 109, 1)'),
                width = 1.5)))

# Edit the layout
layout = dict(
            title = 'Traffic projections<br>vs.<br>traffic simulations',
            yaxis = dict(title = 'Maize throughput (t/year)'),
            annotations=[
                dict(
                    x=1,
                    y=-0.12,
                    showarrow=False,
                    text='Source: Own work',
                    xref='paper',
                    yref='paper'),
                dict(
                    x=2018.7,
                    y=1,
                    showarrow=False,
                    text='Projected traffic',
                    yref='paper',
                    textangle=90,
                    font=dict(
                        size=13)),
                dict(
                    x=2017.3,
                    y=1,
                    showarrow=False,
                    text='Historic traffic',
                    yref='paper',
                    textangle=90,
                    font=dict(
                        size=13))],
            shapes=[
                dict(
                    type='line',
                    x0=2018,
                    y0=-0.05,
                    x1=2018,
                    y1=1.05,
                    yref='paper',
                    line=dict(
                        color='rgb(60, 60, 60)',
                        width=3,
                        dash='dashdot'))]
)

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='Traffic projections vs. simulations')

#### Traffic projections (inc scenario)

In [ ]:
# Visualize each traffic projections
data = []
for i in range(len(traffic_projections)):
    if i == 0:
        data.append(
            go.Scatter(
                x = maize.years,
                y = traffic_projections[i],
                name = 'Traffic projections',
                line = dict(
                    color = ('rgb(215, 215, 215)'),
                    width = 2)))
    else:
        data.append(
            go.Scatter(
                x = maize.years,
                y = traffic_projections[i],
                name = 'Traffic projections',
                showlegend = False,
                line = dict(
                    color = ('rgb(215, 215, 215)'),
                    width = 2)))
    
# Visualize median traffic projections
data.append(
    go.Scatter(
        x = maize.years,
        y = maize.demand,
        name = 'Traffic scenario',
        line = dict(
           color = ('rgb(214, 39, 40)'),
            width = 2)))
    
# Edit the layout
layout = dict(
            title = 'Traffic projections',
            yaxis = dict(title = 'Maize throughput (t/year)'),
            legend=dict(
                x=1.05,
                y=1),
            annotations=[
                dict(
                    x=1,
                    y=-0.12,
                    showarrow=False,
                    text='Source: Own work',
                    xref='paper',
                    yref='paper'),
                dict(
                    x=2018.7,
                    y=1,
                    showarrow=False,
                    text='Projected traffic',
                    yref='paper',
                    textangle=90,
                    font=dict(
                        size=13)),
                dict(
                    x=2017.3,
                    y=1,
                    showarrow=False,
                    text='Historic traffic',
                    yref='paper',
                    textangle=90,
                    font=dict(
                        size=13))],
            shapes=[
                dict(
                    type='line',
                    x0=2018,
                    y0=-0.05,
                    x1=2018,
                    y1=1.05,
                    yref='paper',
                    line=dict(
                        color='rgb(60, 60, 60)',
                        width=3,
                        dash='dashdot'))]
)

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='Traffic projections')

#### Single projection to present quantity

In [ ]:
# Visualize median traffic projection
data = []
data.append(
    go.Scatter(
        x = maize.years,
        y = maize.demand,
        name = 'Traffic projection',
        line = dict(
           color = ('rgb(215, 215, 215)'),
            width =3,
            dash = 'dot')))

# Visualize discounted median traffic projection
discounted_y = []
for year in range(len(maize.demand)):
    discounted_y.append(maize.demand[year] * WACC_factor[year])
data.append(
    go.Scatter(
        x = maize.years,
        y = discounted_y,
        name = 'Discounted projection',
        line = dict(
           color = ('rgb(200, 200, 200)'),
            width =3,)))

# Shade area grey under discounted median traffic projection
discounted_y = []
for year in range(len(maize.demand)):
    discounted_y.append(maize.demand[year] * WACC_factor[year])
data.append(
    go.Scatter(
        x = maize.years[5:],
        y = discounted_y[5:],
        name = 'Integration',
        showlegend = False,
        fill='tozeroy',
        fillcolor='rgba(200, 200, 200, 0.2)',
        line = dict(
            width =0,)))

# Visualize discount factor based on the project's cost of equity
data.append(
    go.Scatter(
        x = maize.years,
        y = WACC_factor,
        name = 'Discount factor',
        yaxis='y2',
        line = dict(
            color = ('rgb(55, 83, 109)'),
            width =3, 
            dash = 'dot')))

# Edit the layout
layout = dict(
            title = 'Discounting traffic projections',
            yaxis = dict(
                title = 'Maize throughput (t/year)',
                side='right',
                ticksuffix='t'
            ),
            yaxis2= dict(
                title='WACC-based discount factor',
                overlaying='y',
                side='left',
                showgrid=False,
                autorange=False,
                range = [0,1]
            ),
            legend=dict(
                x=1.05,
                y=1),
            annotations=[
                dict(
                    x=1,
                    y=-0.12,
                    showarrow=False,
                    text='Source: Own work',
                    xref='paper',
                    yref='paper'),
                dict(
                    x=2018.7,
                    y=0.1,
                    showarrow=False,
                    text='Projected traffic',
                    yref='paper',
                    textangle=90,
                    font=dict(
                        size=13)),
                dict(
                    x=2017.3,
                    y=0.1,
                    showarrow=False,
                    text='Historic traffic',
                    yref='paper',
                    textangle=90,
                    font=dict(
                        size=13))],
            shapes=[
                dict(
                    type='line',
                    x0=2018,
                    y0=-0.05,
                    x1=2018,
                    y1=1.05,
                    yref='paper',
                    line=dict(
                        color='rgb(60, 60, 60)',
                        width=3,
                        dash='dashdot'))]
)

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='Traffic projections')

#### NPQ distribution

In [ ]:
# Add histogram data

# Group data together
color = ['rgba(55, 83, 109, 1)']
hist_data = [df['NPQ']]
group_labels = ['Traffic projections']

# Create distplot with curve_type set to 'normal'
fig = ff.create_distplot(hist_data, group_labels, show_hist=False, colors=color)

fig['layout'].update(
    autosize=False,
    showlegend=False,
    xaxis=dict(
        title='Cumulative traffic volumes in present quantity terms (tonnes)',
        ticksuffix='t'
    ),
    yaxis=dict(
        title='Probability distribution (%)',
        autorange=True,
        showgrid=True,
        zeroline=True,
        showline=True,
        ticks='',
        showticklabels=False
    ),
    annotations=[
        dict(
            x=0.5,
            y=1.25,
            showarrow=False,
            text='Distribution of traffic projections',
            xref='paper',
            yref='paper',
            font=dict(
                size=16)
        ),
        dict(
            x=np.median(df['NPQ'])-70000,
            y=0.86,
            showarrow=False,
            text='Representative projection',
            yref='paper',
            textangle=90,
            font=dict(
                size=12)
        ),
        dict(
            x=1,
            y=-0.12,
            showarrow=False,
            text='Source: Own work',
            xref='paper',
            yref='paper'
        )],
    shapes=[dict(type='line',
                 x0=np.median(df['NPQ']),
                 y0=0.36,
                 x1=np.median(df['NPQ']),
                 y1=1,
                 yref='paper',
                 line=dict(
                     color='rgb(214, 39, 40)',
                     width=2,
                     dash='dot'))],
)

# Plot!
py.iplot(fig, filename='Traffic projection distribution')

### Timing of investments

In [13]:
colors = ['rgba(55, 83, 109, 1)', 'rgba(55, 83, 109, 0.8)', 'rgba(55, 83, 109, 0.5)', 'rgb(129, 180, 179, 1)', 
          'rgb(79, 127, 127, 1)', 'rgb(110,110,110)']

x = maize.years[5:]
y = maize.demand[5:]
capacity_ahead_of_demand = [1400000, 1400000, 1400000, 1400000, 1400000, 1700000, 1700000, 1700000, 1700000, 1700000, 1930000, 
                            1930000, 1930000, 1930000, 1930000, 2200000, 2200000, 2200000, 2200000, 2200000]
demand_ahead_of_capacity = [1350000, 1350000, 1350000, 1350000, 1350000, 1550000, 1550000, 1550000, 1550000, 1550000, 1880000,
                            1880000, 1880000, 1880000, 1880000, 2150000, 2150000, 2150000, 2150000, 2150000]

trace1 = {
  'x': x,
  'y': capacity_ahead_of_demand,
  'name': 'Capacity ahead<br>of demand',
  'type': 'bar',
  'marker': dict(color=colors[2])
};

trace2 = {
  'x': x,
  'y': demand_ahead_of_capacity,
  'name': 'Demand ahead<br>of capacity',
  'type': 'bar',
  'marker': dict(color=colors[0])
};

trace3 = go.Scatter(
    name='Traffic projection',
    x=x,
    y=y,
    mode='lines',
    line=dict(
        color='rgb(214, 39, 40)', 
        width=3
    )
)

data = [trace1, trace2, trace3];
layout = {
    'xaxis': dict(
            tick0=0,
            dtick=1,
            tickangle=315),
    'yaxis': {'title': 'Terminal capacity (t/year)',
            'range':[1000000,2400000]},
    'title': 'Timing investments',
    'legend': dict(x=1.1,
                 y=1),
    'annotations':[
                    dict(
                        x=0.4,
                        y=0.53,
                        showarrow=False,
                        text='Demand',
                        font=dict(
                                color='rgb(214, 39, 40)',
                                size=14),
                        xref='paper',
                        yref='paper')],
    'bargap': 0.15,
    'bargroupgap' :0.1,
};

py.iplot({'data': data, 'layout': layout}, filename='barmode-relative')